
# MMF1921/MIE377 - Backtesting Template

The purpose of this program is to provide a template with which to develop Project 2. The project requires you to test different models  (and/or different model combinations) to create an asset management algorithm.

This template will be used by the instructor and TA to assess your trading algorithm using different datasets.

# PLEASE DO NOT MODIFY THIS TEMPLATE (for Project submission purposes)

# 1. Read input files

In [1]:
import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from services.project_function import *
import pandas as pd

adjClose = pd.read_csv("MIE377_AssetPrices_3.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns_3.csv", index_col=0)

In [2]:
adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

In [3]:
# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];


In [4]:
#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

In [5]:
# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]
returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

In [6]:
assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

In [7]:
adjClose = pd.read_csv("MIE377_AssetPrices_2.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns_2.csv", index_col=0)

adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];

#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index

# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]
returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]


# 3. Results

In [8]:
def run_strategy(num_periods,lambda_RP, lamda_S, z, model, cal, file_1, file_2):

    adjClose = pd.read_csv(file_1, index_col=0)
    factorRet = pd.read_csv(file_2, index_col=0)

    adjClose.index = pd.to_datetime(adjClose.index)
    factorRet.index = pd.to_datetime(factorRet.index)

    # Initial budget to invest ($100,000)
    initialVal = 100000

    # Length of investment period (in months)
    investPeriod = 6

    # divide the factor returns by  100
    factorRet = factorRet/100

    #rf and factor returns
    riskFree = factorRet['RF']
    factorRet = factorRet.loc[:,factorRet.columns != 'RF'];

    #Identify the tickers and the dates
    tickers = adjClose.columns
    dates   = factorRet.index

    # Calculate the stocks monthly excess returns
    # pct change and drop the first null observation
    returns = adjClose.pct_change(1).iloc[1:, :]
    returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
    # Align the price table to the asset and factor returns tables by discarding the first observation.
    adjClose = adjClose.iloc[1:,:]

    assert adjClose.index[0] == returns.index[0]
    assert adjClose.index[0] == factorRet.index[0]



    testStart = returns.index[0] + pd.offsets.DateOffset(years=cal)
    investPeriod = 5
    testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
    calEnd = testStart - pd.offsets.DateOffset(days=1)

    NoPeriods = math.ceil((returns.index[-1].to_period('M') - testStart.to_period('M')).n / investPeriod)
    n = len(tickers)

    x = np.zeros([n, NoPeriods])
    x0 = np.zeros([n, NoPeriods])
    currentVal = np.zeros([NoPeriods, 1])
    turnover = np.zeros([NoPeriods, 1])
    toDay = 0
    portfValue = []

    for t in range(NoPeriods):
        periodReturns = returns[returns.index <= calEnd]
        periodFactRet = factorRet[factorRet.index <= calEnd]

        current_price_idx = (calEnd - pd.offsets.DateOffset(months=1) <= adjClose.index)&(adjClose.index <= calEnd)
        currentPrices = adjClose[current_price_idx]

        periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
        periodPrices = adjClose[periodPrices_idx]

        if len(periodPrices) != investPeriod or len(currentPrices) != 1:
            break  # Skip this period if data is missing

        if t == 0:
            currentVal[0] = initialVal
        else:
            currentVal[t] = currentPrices @ NoShares.values.T
            x0[:,t] = currentPrices.values * NoShares.values / currentVal[t]

        x[:,t] = grid_search_function(periodReturns, periodFactRet, num_periods,lambda_RP, lamda_S, z, model)

        if t > 0:
            turnover[t] = np.sum(np.abs(x[:,t] - x0[:,t]))

        NoShares = x[:,t] * currentVal[t] / currentPrices

        fromDay = toDay
        toDay = toDay + len(periodPrices)

        portfValue.append(periodPrices @ NoShares.values.T)

        testStart = testStart + pd.offsets.DateOffset(months=investPeriod)
        testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
        calEnd = testStart - pd.offsets.DateOffset(days=1)

    if len(portfValue) == 0:
        return -np.inf, np.nan  # Invalid run

    portfValue = pd.concat(portfValue, axis=0)
    portfRets = portfValue.pct_change(1).iloc[1:,:]
    portfExRets = portfRets.subtract(riskFree[(riskFree.index >= portfRets.index[0]) & (riskFree.index <= portfRets.index[-1])], axis=0)

    SR = ((portfExRets + 1).apply(gmean, axis=0) - 1) / portfExRets.std()
    avgTurnover = np.mean(turnover[1:])

    return SR[0], avgTurnover


In [9]:
best_results = {}


for model in range(3):
    best_ratio = -np.inf
    best_params = {}

    # Model 0 (Risk Parity)
    if model == 0:
        # Not neccarry to loop through lambda_s and z for this model
        lamda_S = "N/A"
        z = "N/A"
        for lambda_RP in [0.25, 0.5, 0.75]:  # example value
            # loop through 2 different amount of training months to see which paramters is better
            for num_periods in [30,36]:
                SRs = []
                turnovers = []
                # Loop through both datasets for training
                for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"], 
                            ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                    # iterate through different training windows
                    for cal in [3, 5, 8]:
                        #Call function to perform test and run the model
                        print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                        SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP, lamda_S=None, z=None,
                                                    model=model, cal=cal, file_1=file[0], file_2=file[1])
                        print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                        # Save resutls from this iteration of the parametrs
                        SRs.append(SR)
                        turnovers.append(turnover)

                # Calculate metrics from combination of parameters
                avg_SR = np.mean(SRs)
                avg_turnover = np.mean(turnovers)
                avg_ratio = avg_SR / avg_turnover

                # Check if the model is better and if it is update the saved "Best model and paramters"
                if avg_ratio > best_ratio:
                    best_ratio = avg_ratio
                    best_params = {
                        'model': model,
                        'lambda_RP': lambda_RP,
                        'lambda_S': lamda_S,
                        'z': z,
                        'num_periods': num_periods,
                        'SR': avg_SR,
                        'turnover': avg_turnover
                    }

    # Model 1 (Sharpe)
    elif model == 1:
        # Do not need to iterate through lambda_rp for sharpe model
        lambda_RP = "N/A"
        # Do not need to iterate through sampling weigths for sharpe
        z = "N/A"
        # Iterate trhough different values of lambd_s
        for lamda_S in [8, 12, 16]:
            # Iterate through different number of training lengths
            for num_periods in [30, 36]:
                SRs = []
                turnovers = []
                # Iterate through different training datasets 1 and 2
                for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"],
                             ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                    # iterate through different training windows
                    for cal in [3, 5, 8]:
                        print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                        SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=None, lamda_S=lamda_S, z=None,
                                                    model=model, cal=cal, file_1=file[0], file_2=file[1])
                        print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                        # Save results from iteration 
                        SRs.append(SR)
                        turnovers.append(turnover)

                #Aggregate all results from combination of paramters
                avg_SR = np.mean(SRs)
                avg_turnover = np.mean(turnovers)
                avg_ratio = avg_SR / avg_turnover

                # Check if parameters yeild better sharpe to turnover ratio
                # If yes then replace the saved paramters
                if avg_ratio > best_ratio:
                    best_ratio = avg_ratio
                    best_params = {
                        'model': model,
                        'lambda_RP': lambda_RP,
                        'lambda_S': lamda_S,
                        'z': z,
                        'num_periods': num_periods,
                        'SR': avg_SR,
                        'turnover': avg_turnover
                    }

    # Model 2 (Mixed)
    elif model == 2:
        #Iterate trhough different values of lambda_rp
        for lambda_RP in [0.25, 0.5, 0.75]:
            #Iteration through different values of lambda_s
            for lamda_S in [8, 12, 16]:
                #Iterate through different values of z or sampling weighted value
                for z in [0.25, 0.5, 0.75]:
                    #Iterate through numnber of training periods available
                    for num_periods in [30, 36]:
                        SRs = []
                        turnovers = []
                        # Iterate through training datasets
                        for file in [["MIE377_AssetPrices_1.csv", "MIE377_FactorReturns_1.csv"],
                                     ["MIE377_AssetPrices_2.csv", "MIE377_FactorReturns_2.csv"]]:
                            #Iterate through data training windows
                            for cal in [3, 5, 8 ]:
                                print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                                #Run model
                                SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP,
                                                            lamda_S=lamda_S, z=z, model=model, cal=cal,
                                                            file_1=file[0], file_2=file[1])
                                print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                                #Save results of model with chosen params in specific window
                                SRs.append(SR)
                                turnovers.append(turnover)

                        #Aggregate reuslts from combination of params
                        avg_SR = np.mean(SRs)
                        avg_turnover = np.mean(turnovers)
                        avg_ratio = avg_SR / avg_turnover

                        # Check if parameters yeild better sharpe to turnover ratio
                        # If yes then replace the saved paramters
                        if avg_ratio > best_ratio:
                            best_ratio = avg_ratio
                            best_params = {
                                'model': model,
                                'lambda_RP': lambda_RP,
                                'lambda_S': lamda_S,
                                'z': z,
                                'num_periods': num_periods,
                                'SR': avg_SR,
                                'turnover': avg_turnover
                            }

    best_results[model] = best_params


Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=3
--> SR = 0.2136, Turnover = 0.1182

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=5
--> SR = 0.1724, Turnover = 0.1185

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=8
--> SR = 0.3221, Turnover = 0.0990

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=3
--> SR = 0.2590, Turnover = 0.0913

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=5
--> SR = 0.2088, Turnover = 0.1039

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=30, cal=8
--> SR = 0.1847, Turnover = 0.1015

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=36, cal=3
--> SR = 0.2156, Turnover = 0.1114

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=36, cal=5
--> SR = 0.1761, Turnover = 0.1121

Testing model 0, lambda_RP=0.25, lambda_S=N/A, z=N/A, num_periods=36, cal=8
--> SR = 0.3268, Turnover =

/opt/anaconda3/envs/mie377/lib/python3.10/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


--> SR = 0.1415, Turnover = 0.3394

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=36, cal=8
--> SR = 0.1059, Turnover = 0.3798

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=3
--> SR = 0.2391, Turnover = 0.3749

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=5
--> SR = 0.1787, Turnover = 0.3927

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=8
--> SR = 0.3061, Turnover = 0.3453

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=3
--> SR = 0.2268, Turnover = 0.2960

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=5
--> SR = 0.1499, Turnover = 0.3842

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=30, cal=8
--> SR = 0.1122, Turnover = 0.3772

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=36, cal=3
--> SR = 0.2420, Turnover = 0.2976

Testing model 1, lambda_RP=N/A, lambda_S=12, z=N/A, num_periods=36, cal=5
--> SR = 0.

/opt/anaconda3/envs/mie377/lib/python3.10/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


--> SR = 0.1590, Turnover = 0.2592

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.25, num_periods=36, cal=8
--> SR = 0.1248, Turnover = 0.2863

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=3
--> SR = 0.2336, Turnover = 0.2013

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=5
--> SR = 0.1809, Turnover = 0.2088

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=8
--> SR = 0.3215, Turnover = 0.1803

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=3
--> SR = 0.2449, Turnover = 0.1625

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=5
--> SR = 0.1811, Turnover = 0.2008

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=30, cal=8
--> SR = 0.1496, Turnover = 0.2007

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=36, cal=3
--> SR = 0.2354, Turnover = 0.1610

Testing model 2, lambda_RP=0.25, lambda_S=8, z=0.5, num_periods=36, cal=5
--> SR = 

...

--> SR = 0.1568, Turnover = 0.2554

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.25, num_periods=36, cal=8
--> SR = 0.1217, Turnover = 0.2829

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=3
--> SR = 0.2339, Turnover = 0.1946

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=5
--> SR = 0.1843, Turnover = 0.2006

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=8
--> SR = 0.3189, Turnover = 0.1717

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=3
--> SR = 0.2391, Turnover = 0.1562

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=5
--> SR = 0.1754, Turnover = 0.1938

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=30, cal=8
--> SR = 0.1405, Turnover = 0.1914

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=36, cal=3
--> SR = 0.2360, Turnover = 0.1586

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=36, cal=5
--> SR = 

/opt/anaconda3/envs/mie377/lib/python3.10/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


--> SR = 0.1711, Turnover = 0.1751

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.5, num_periods=36, cal=8
--> SR = 0.1368, Turnover = 0.1901

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=3
--> SR = 0.2234, Turnover = 0.1260

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=5
--> SR = 0.1804, Turnover = 0.1264

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=8
--> SR = 0.3181, Turnover = 0.1050

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=3
--> SR = 0.2435, Turnover = 0.0949

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=5
--> SR = 0.1865, Turnover = 0.1100

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=30, cal=8
--> SR = 0.1532, Turnover = 0.1067

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=3
--> SR = 0.2252, Turnover = 0.1087

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=5
--

/opt/anaconda3/envs/mie377/lib/python3.10/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


--> SR = 0.1844, Turnover = 0.1037

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=8
--> SR = 0.1512, Turnover = 0.1059

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=3
--> SR = 0.2395, Turnover = 0.2791

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=5
--> SR = 0.1837, Turnover = 0.2928

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=8
--> SR = 0.3142, Turnover = 0.2543

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=3
--> SR = 0.2332, Turnover = 0.2248

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=5
--> SR = 0.1630, Turnover = 0.2880

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=30, cal=8
--> SR = 0.1269, Turnover = 0.2831

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=36, cal=3
--> SR = 0.2424, Turnover = 0.2239

Testing model 2, lambda_RP=0.75, lambda_S=12, z=0.25, num_periods=36,

In [10]:
# Prepare data for a nice table
table_data = []
for model, result in best_results.items():
    table_data.append({
        'Model': model,
        'lambda_RP': result.get('lambda_RP', 'N/A'),
        'lambda_S': result.get('lambda_S', 'N/A'),
        'z': result.get('z', 'N/A'),
        'num_periods': result['num_periods'],
        'SR': result['SR'],
        'Turnover': result['turnover']
    })

# Convert to DataFrame for better readability
df_results = pd.DataFrame(table_data)

# Display the results in a table format
print("\nBest Results Summary:")
print(df_results)



Best Results Summary:
   Model lambda_RP lambda_S     z  num_periods        SR  Turnover
0      0      0.75      N/A   N/A           36  0.218721  0.087885
1      1       N/A        8   N/A           36  0.208535  0.310692
2      2      0.75        8  0.75           36  0.219707  0.101535


In [9]:
best_results = {}

'''
This is the testing function/ loop:
1. Take optimal parameters for each method from above training process
2. Set those values in the loops
3. Execute testing procedure using exact same logic as training but with test dataset 3 
    and with the chosen paramters from above.
'''



for model in range(3):
    best_ratio = -np.inf
    best_params = {}

    # Model 0 (Risk Parity)
    if model == 0:
        lambda_RP = 0.75 # example value
        lamda_S = "N/A"
        z = "N/A"
        for num_periods in [36]:
            SRs = []
            turnovers = []
            for file in [["MIE377_AssetPrices_3.csv", "MIE377_FactorReturns_3.csv"]]:
                for cal in [3, 5, 8]:
                    print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                    SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP, lamda_S=None, z=None,
                                                model=model, cal=cal, file_1=file[0], file_2=file[1])
                    print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                    SRs.append(SR)
                    turnovers.append(turnover)

            avg_SR = np.mean(SRs)
            avg_turnover = np.mean(turnovers)
            avg_ratio = avg_SR / avg_turnover

            if avg_ratio > best_ratio:
                best_ratio = avg_ratio
                best_params = {
                    'model': model,
                    'lambda_RP': lambda_RP,
                    'lambda_S': lamda_S,
                    'z': z,
                    'num_periods': num_periods,
                    'SR': avg_SR,
                    'turnover': avg_turnover
                }

    # Model 1 (Sharpe)
    elif model == 1:
        lambda_RP = "N/A"
        z = "N/A"
        for lamda_S in [8]:
            for num_periods in [36]:
                SRs = []
                turnovers = []
                for file in [["MIE377_AssetPrices_3.csv", "MIE377_FactorReturns_3.csv"]]:
                    for cal in [3, 5, 8]:
                        print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                        SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=None, lamda_S=lamda_S, z=None,
                                                    model=model, cal=cal, file_1=file[0], file_2=file[1])
                        print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                        SRs.append(SR)
                        turnovers.append(turnover)

                avg_SR = np.mean(SRs)
                avg_turnover = np.mean(turnovers)
                avg_ratio = avg_SR / avg_turnover

                if avg_ratio > best_ratio:
                    best_ratio = avg_ratio
                    best_params = {
                        'model': model,
                        'lambda_RP': lambda_RP,
                        'lambda_S': lamda_S,
                        'z': z,
                        'num_periods': num_periods,
                        'SR': avg_SR,
                        'turnover': avg_turnover
                    }

    # Model 2 (Mixed)
    elif model == 2:
        for lambda_RP in [0.75]:
            for lamda_S in [8]:
                for z in [0.75]:
                    for num_periods in [36]:
                        SRs = []
                        turnovers = []
                        for file in [["MIE377_AssetPrices_3.csv", "MIE377_FactorReturns_3.csv"]]:
                            for cal in [3,5,8]:
                                print(f"\nTesting model {model}, lambda_RP={lambda_RP}, lambda_S={lamda_S}, z={z}, num_periods={num_periods}, cal={cal}")
                                SR, turnover = run_strategy(num_periods=num_periods, lambda_RP=lambda_RP,
                                                            lamda_S=lamda_S, z=z, model=model, cal=cal,
                                                            file_1=file[0], file_2=file[1])
                                print(f"--> SR = {SR:.4f}, Turnover = {turnover:.4f}")
                                SRs.append(SR)
                                turnovers.append(turnover)

                        avg_SR = np.mean(SRs)
                        avg_turnover = np.mean(turnovers)
                        avg_ratio = avg_SR / avg_turnover

                        if avg_ratio > best_ratio:
                            best_ratio = avg_ratio
                            best_params = {
                                'model': model,
                                'lambda_RP': lambda_RP,
                                'lambda_S': lamda_S,
                                'z': z,
                                'num_periods': num_periods,
                                'SR': avg_SR,
                                'turnover': avg_turnover
                            }

    best_results[model] = best_params

# Prepare data for a nice table
table_data = []
for model, result in best_results.items():
    table_data.append({
        'Model': model,
        'lambda_RP': result.get('lambda_RP', 'N/A'),
        'lambda_S': result.get('lambda_S', 'N/A'),
        'z': result.get('z', 'N/A'),
        'num_periods': result['num_periods'],
        'SR': result['SR'],
        'Turnover': result['turnover']
    })

# Convert to DataFrame for better readability
df_results = pd.DataFrame(table_data)

# Display the results in a table format
print("\nBest Results Summary:")
print(df_results)


Testing model 0, lambda_RP=0.75, lambda_S=N/A, z=N/A, num_periods=36, cal=3
--> SR = 0.2750, Turnover = 0.1248

Testing model 0, lambda_RP=0.75, lambda_S=N/A, z=N/A, num_periods=36, cal=5
--> SR = 0.2108, Turnover = 0.1292

Testing model 0, lambda_RP=0.75, lambda_S=N/A, z=N/A, num_periods=36, cal=8
--> SR = 0.1876, Turnover = 0.1360

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=36, cal=3
--> SR = 0.3213, Turnover = 0.3260

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=36, cal=5
--> SR = 0.2539, Turnover = 0.3152

Testing model 1, lambda_RP=N/A, lambda_S=8, z=N/A, num_periods=36, cal=8
--> SR = 0.2050, Turnover = 0.3599

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=3
--> SR = 0.2933, Turnover = 0.1276

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=5
--> SR = 0.2258, Turnover = 0.1331

Testing model 2, lambda_RP=0.75, lambda_S=8, z=0.75, num_periods=36, cal=8
--> SR = 0.1965, Turnover = 0.1304

Bes